In [1]:
from IPython.display import clear_output

!pip install datasets transformers rouge_score nltk

# !pip install -q sentencepiece
# !pip install rouge-score

clear_output()

In [9]:
import os
import re
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

# pytorch dataset types
import datasets
from datasets.dataset_dict import DatasetDict
from datasets import Dataset, load_metric

import tensorflow as tf
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration # pegasus
from transformers import BartTokenizer, TFBartForConditionalGeneration # bart
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = "/content/gdrive/MyDrive/Classes/W266_NLP/w266_reddit_summarization/data/tensorflow_datasets/reddit/1.0.0"
os.chdir(data_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# define train/test split from the data. but don't read the data yet.
file_names = os.listdir(data_dir)
metadata_files = [i for i in file_names if re.search('^(?!reddit-train)', i)]

data_files = np.array([i for i in file_names if re.search('^reddit-train', i)])
split_ind = np.array([int(x[22:27]) % 5 for x in data_files])
test = data_files[split_ind == 0]
train = data_files[split_ind != 0]

In [5]:
# shorten the data for now, to make everything simpler
valid = train[4:5]
train = train[:3]
test = test[:2]

In [30]:
start = time.time()
def parse_file(serialized_example, return_xy=False):

  file_scruct = {
      'author': tf.io.FixedLenFeature([], tf.string),
      'body': tf.io.FixedLenFeature([], tf.string),
      'content': tf.io.FixedLenFeature([], tf.string),
      'id': tf.io.FixedLenFeature([], tf.string),
      'normalizedBody': tf.io.FixedLenFeature([], tf.string),
      'subreddit': tf.io.FixedLenFeature([], tf.string),
      'subreddit_id': tf.io.FixedLenFeature([], tf.string),
      'summary': tf.io.FixedLenFeature([], tf.string),
  }

  example_parsed = tf.io.parse_single_example(serialized_example, file_scruct)
  return example_parsed


# parse
train_tf = tf.data.TFRecordDataset(train).map(parse_file)
valid_tf = tf.data.TFRecordDataset(valid).map(parse_file)
test_tf = tf.data.TFRecordDataset(test).map(parse_file)

decode_string = np.vectorize(lambda x: x.decode('utf-8'))

# load data into memory into dictionary. 
# figure out how to bypass this
## we're doing tf -> np -> pt. Want to go tf -> pt
def tf_to_dict(tf_item):
  dataset_dict = {'content': [], 'summary': [], 'subreddit': []}
  for item in tf_item: 
    dataset_dict['content'].append(str(decode_string(item['content'].numpy())))
    dataset_dict['summary'].append(str(decode_string(item['summary'].numpy())))
    dataset_dict['subreddit'].append(str(decode_string(item['subreddit'].numpy())))
  return dataset_dict
# def tf_to_dict(tf_item):
#   dataset_dict = {'content': [], 'summary': [], 'subreddit': []}
#   for item in tf_item: 
#     dataset_dict['content'].append(item['content'])
#     dataset_dict['summary'].append(item['summary'])
#     dataset_dict['subreddit'].append(item['subreddit'])
#   return dataset_dict

# convert to pytorch Dataset datatype
all_data = DatasetDict({
    'train': Dataset.from_dict(tf_to_dict(train_tf)), 
    'valid': Dataset.from_dict(tf_to_dict(valid_tf)), 
    'test': Dataset.from_dict(tf_to_dict(test_tf))})

seconds_elapsed = time.time() - start
print(f"{seconds_elapsed/60} minutes elapsed")

0.7704395135243733 minutes elapsed


In [31]:
# visualize some examples
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


show_random_elements(all_data['train'])

# Load BART

In [32]:
from transformers import BartTokenizer, TFBartForConditionalGeneration
model = TFBartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# load the metric we'll use which is rouge
metric = load_metric('rouge')

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [34]:
# one example, no fine tuning
inputs = tokenizer(all_data['train']['content'][0], max_length=1024, truncation=True, return_tensors="tf", padding='max_length')
summary_ids = model.generate(inputs["input_ids"], 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=1,
                              max_length=150)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False), compact=True)

['Here is something Olympic middle distance runner Nick Willis wrote about '
 'fellow Kiwi Sean Adams (Pitt)  .. Basketball fans in New Zealand look once '
 'again to have a kiwhi who will be able play for the Detroit Pistons, and '
 'become one of their own on American hardwood! In terms or NBA draft history '
 'though – it’s been quite some time since we had an Australian player go pro; '
 'as far back at least 15 years ago when Kirk Penny was drafted by Miami '
 'Heat…and up until now I would say that aside from Steven “The Kid From '
 'Rotorua″ Smith ,New Zealander basketball fan here has never seen anything '
 'like this before: Up till recently–in fact over 20 seasons old-that']


In [81]:
# tokenize data to train the model
def tokenize_fun(x):
  return tokenizer(x['content'], max_length=1024, truncation=True, padding='max_length') #return_tensors="tf"

all_data_tokenized = all_data.map(tokenize_fun)

  0%|          | 0/45097 [00:00<?, ?ex/s]

  0%|          | 0/15032 [00:00<?, ?ex/s]

  0%|          | 0/30066 [00:00<?, ?ex/s]

In [82]:
all_data_tokenized

DatasetDict({
    train: Dataset({
        features: ['content', 'summary', 'subreddit', 'input_ids', 'attention_mask'],
        num_rows: 45097
    })
    valid: Dataset({
        features: ['content', 'summary', 'subreddit', 'input_ids', 'attention_mask'],
        num_rows: 15032
    })
    test: Dataset({
        features: ['content', 'summary', 'subreddit', 'input_ids', 'attention_mask'],
        num_rows: 30066
    })
})

In [35]:
# tokenize it differently so it also tokenizes the summary
max_input_length = 1024
max_target_length = 128

model_checkpoint = 'facebook/bart-large'

# if we're using t5, append "summarize: " in front
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

def preprocess_function(examples, xvar='content', yvar='summary', max_input_length=1024, max_target_length=128):
    inputs = [prefix + doc for doc in examples[xvar]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples[yvar], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# now tokenize our data
all_data_tokenized2 = all_data.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f245405a8c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/31 [00:00<?, ?ba/s]

In [36]:
# inspect. so now the input_ids are tokenized inputs. and the labels are tokenized summaries. 
all_data_tokenized2['train']

Dataset({
    features: ['content', 'summary', 'subreddit', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 45097
})

In [44]:
# set params
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

# use collator to convert our PT dataset to TF dataset
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

# NOTE, I'M TRUNCATING THE DATA TO ONLY 10 OBS TO SPEED THIS UP
# REMOVE THE .select() LATER
train_dataset = all_data_tokenized2["train"].select(range(10)).to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = all_data_tokenized2["valid"].select(range(10)).to_tf_dataset(
    batch_size=8,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)

In [48]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
# model.compile(optimizer=optimizer)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)


model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

ResourceExhaustedError: ignored